In [2]:
folder = "/Users/flint/Data/kaggle/books/book-recommender/"
metadata_file = "collaborative_book_metadata.csv"
ratings_file = "collaborative_books_df.csv"

## I/O generico da file

In [3]:
metadata_f = folder + metadata_file
ratings_f = folder + ratings_file
reader = open(ratings_f, 'r')
content = reader.readlines()
reader.close()

## Pandas

In [4]:
import pandas as pd 
import numpy as np 

In [5]:
c1 = {
    'nome': 'gino', 'cognome': 'bianchi', 'eta': 34}
c2 = {'nome': 'maria', 'cognome': 'verdi', 
    'eta': 24, 'indirizzo': 'torino', 'civile': 'nubile'}
c3 = {'nome': 'ettore', 'cognome': 'rossi', 
    'eta': 44, 'indirizzo': 'milano'}
clients = [c1, c2, c3]

df = pd.DataFrame(clients)

In [6]:
df 

,nome,cognome,eta,indirizzo,civile
0,gino,bianchi,34,NaN,NaN
1,maria,verdi,24,torino,nubile
2,ettore,rossi,44,milano,NaN


In [7]:
df.groupby('indirizzo')['eta'].mean()

indirizzo
milano    44.0
torino    24.0
Name: eta, dtype: float64

In [8]:
numbers = np.random.randint(0, 10, size=10000)
output = {}
for n in numbers:
    if n in output.keys():
        output[n] = output[n] + 1
    else:
        output[n] = 1


In [9]:
o = pd.Series(output)

In [10]:
o.sort_values(ascending=False)

9    1075
3    1019
7    1017
4    1010
0     998
2     987
6     987
1     976
5     975
8     956
dtype: int64

## I/O con Pandas

In [11]:
ratings = pd.read_csv(ratings_f, index_col=0)
metadata = pd.read_csv(metadata_f, index_col=0)

In [12]:
metadata.head(3)

,book_id,title,image_url,url,num_pages,ratings_count,description,genre,name,book_id_mapping
0,5899779,Pride and Prejudice and Zombies Pride and Prej...,https://images.gr-assets.com/books/1320449653m...,https://www.goodreads.com/book/show/5899779-pr...,320,105537,The New York Times Best Seller is now a major ...,"['fantasy, paranormal', 'romance', 'fiction', ...",Jane Austen,808
1,872333,Blue Bloods Blue Bloods 1,https://images.gr-assets.com/books/1322281515m...,https://www.goodreads.com/book/show/872333.Blu...,302,117633,"When the Mayflower set sail in 1620, it carrie...","['young-adult', 'fantasy, paranormal', 'romanc...",Melissa de la Cruz,217
2,15507958,Me Before You Me Before You 1,https://images.gr-assets.com/books/1357108762m...,https://www.goodreads.com/book/show/15507958-m...,369,609327,Louisa Clark is an ordinary young woman living...,"['romance', 'fiction']",Jojo Moyes,385


In [22]:
metadata.iloc[0]['genre']

"['fantasy, paranormal', 'romance', 'fiction', 'history, historical fiction, biography', 'young-adult', 'mystery, thriller, crime']"

In [13]:
ratings.tail(3)

,title,book_id,user_id_mapping,book_id_mapping,Predicted Rating,Actual Rating
196293,Coraline,17061,31383,106,3.6,3
196294,Coraline,17061,33833,106,3.7,5
196295,Coraline,17061,61413,106,3.8,4


### Note
1. Libri
   1. genere dei libri 
   2. autore 
   3. numero pagine (da verificare tramite correlazione)
2. Utenti
   1. simili se apprezzano le stesse cose (libri)
   2. preferenze degli utenti rispetto ai generi 
   3. preferenze degli utenti rispetto agli autori

In [14]:
books_data = ['book_id_mapping', 'title', 'num_pages', 'genre', 'name']
ratings_data = ['book_id_mapping', 'user_id_mapping', 'Actual Rating']
meta_df = metadata[books_data]
ratings_df = ratings[ratings_data]

In [15]:
meta_df

,book_id_mapping,title,num_pages,genre,name
0,808,Pride and Prejudice and Zombies Pride and Prej...,320,"['fantasy, paranormal', 'romance', 'fiction', ...",Jane Austen
1,217,Blue Bloods Blue Bloods 1,302,"['young-adult', 'fantasy, paranormal', 'romanc...",Melissa de la Cruz
2,385,Me Before You Me Before You 1,369,"['romance', 'fiction']",Jojo Moyes
3,192,Sharp Objects,254,"['mystery, thriller, crime', 'fiction']",Gillian Flynn
4,873,The Way of Kings The Stormlight Archive 1,1007,"['fantasy, paranormal', 'fiction']",Brandon Sanderson
...,...,...,...,...,...
91,356,How to Be a Woman,312,"['non-fiction', 'history, historical fiction, ...",Caitlin Moran
92,708,Persepolis The Story of a Childhood Persepolis 1,153,"['comics, graphic', 'non-fiction', 'history, h...",Mattias Ripa
93,175,The City of Ember Book of Ember 1,270,"['young-adult', 'fantasy, paranormal', 'fictio...",Jeanne DuPrau
94,374,Wicked The Life and Times of the Wicked Witch ...,409,"['fantasy, paranormal', 'fiction', 'young-adul...",Douglas Smith


In [16]:
genre_values = meta_df['genre']

In [31]:
mapping = []

for k, v in genre_values.items():
    x = v.replace('[', '').replace(']', '')
    x = x.split("', '")
    x[0] = x[0].replace("'", '')
    x[-1] = x[-1].replace("'", '')
    for genre in x:
        mapping.append((meta_df.loc[k].book_id_mapping, genre))

In [32]:
mapping[:3]

[(808, 'fantasy, paranormal'), (808, 'romance'), (808, 'fiction')]

In [33]:
map_genre = pd.DataFrame(mapping, columns=['book', 'genre'])
map_genre

,book,genre
0,808,"fantasy, paranormal"
1,808,romance
2,808,fiction
3,808,"history, historical fiction, biography"
4,808,young-adult
...,...,...
372,508,"fantasy, paranormal"
373,508,fiction
374,508,young-adult
375,508,children


In [34]:
meta_df = meta_df[['book_id_mapping', 'title', 'num_pages', 'name']]

In [35]:
meta_df.head(2)

,book_id_mapping,title,num_pages,name
0,808,Pride and Prejudice and Zombies Pride and Prej...,320,Jane Austen
1,217,Blue Bloods Blue Bloods 1,302,Melissa de la Cruz


In [36]:
ratings_df.head(2)

,book_id_mapping,user_id_mapping,Actual Rating
0,299,1537,5
1,299,23039,3


In [37]:
map_genre.head(10)

,book,genre
0,808,"fantasy, paranormal"
1,808,romance
2,808,fiction
3,808,"history, historical fiction, biography"
4,808,young-adult
5,808,"mystery, thriller, crime"
6,217,young-adult
7,217,"fantasy, paranormal"
8,217,romance
9,217,fiction


## Interazione con database relazionale (sqlite)

In [38]:
from sqlalchemy import create_engine, text 

In [39]:
connection_string = "sqlite:////Users/flint/Data/kaggle/books/book-recommender/booksdb2.sql"
engine = create_engine(connection_string)
conn = engine.connect()
ratings_df.to_sql('ratings', con=conn, if_exists='replace', index=False)
meta_df.to_sql('books', con=conn, if_exists='replace', index=False)
map_genre.to_sql('genre', con=conn, if_exists='replace', index=False)
conn.close()